In [1]:
#scrape urban turf
%pip install pandas
%pip install requests
import requests
%pip install beautifulsoup4
%pip install geopy
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import certifi

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\annas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\annas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\annas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\annas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
page = requests.get(
    "https://dc.urbanturf.com/pipeline?search_string=&project_type=0&status=0&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes&limit=1000")
soup = BeautifulSoup(page.content, 'html.parser')
page_body = soup.body

labels = [item for item in page_body.findAll("p")]
list_lab = [i.text.strip() for i in labels][1:] 
list_lab = [word.replace('\n','') for word in list_lab]

torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using 'torep' value as keys
pattern = re.compile("|".join(torep.keys())) #joins with "|"

desc = []

for i in range(0, len(list_lab)):
    a = list_lab[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #substitute the pattern in each value of the list with ""
    desc.append(text)

seq = range(0, len(desc))

urban_turf= pd.DataFrame(
    {"Address": [desc[i] for i in seq[0::5]],
     "Neighborhood": [desc[i] for i in seq[1::5]],
     "Project.Type":[desc[i] for i in seq[2::5]],
     "Status":[desc[i] for i in seq[3::5]],
     "Size":[desc[i] for i in seq[4::5]],
     "Pipeline Link": None,
     'Neighborhood:':None,
     'Pipeline':None,
     'Project type:': None, 
     'No. of units:': None, 
     'Types of units:': None, 
     'Unit sizes:':None, 
     'Amenities:':None, 
     'Pricing:':None, 
     'Website:':None, 
     'Architect:':None, 
     'Last updated:':None, 
     'Address:':None, 
     'Status:':None, 
     'First move-ins:':None, 
     "Latitude":None, 
     "Longitude":None})    

#can I get the pipeline link from scraping above?

def search_pipe_link(row):
    ut_address_col = str(row['Address'])
    projecttype_col = str(row['Project.Type'])
    status_col = str(row['Status'])
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + ut_address_col.replace(" ", "_" ) + "&project_type=" + projecttype_col.replace(" ", "+").replace("&", "%26") + "&status=" + status_col.replace(",", "%2C").replace(" ", "+") + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    time.sleep(1.25)
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

urban_turf['Pipeline Link'] = urban_turf.apply(search_pipe_link, axis=1)
urban_turf = urban_turf.dropna(subset=['Pipeline Link']) #would like to encorporate into function
urban_turf = urban_turf.reset_index(drop=True)
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

def scrape_pipelinelink(row):
    page = requests.get(row["Pipeline Link"])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]

    key = [txt[i][0] for i in list(range(0,len(txt)))]
    txt = [txt[i][1] for i in list(range(0,len(txt)))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])

    for key, txt in df_dictionary.items():
        if key in column_names:
            row[key] = df_dictionary[key][0]

urban_turf.apply(scrape_pipelinelink, axis=1)

urban_turf['No. of units:'] = [x.replace(" ", "").replace("</p>", "") for x in urban_turf['No. of units:']]
urban_turf.loc[urban_turf['Architect:'].notnull(), 'Architect:'] = [x.replace("\t", "").replace("</p>", "") for x in urban_turf.loc[urban_turf['Architect:'].notnull(), 'Architect:']]
urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'] = [re.search(r'".*\"\sstyle', x).group(0).replace('" style', "").replace('"', "") for x in urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:']]
urban_turf.loc[urban_turf['Status:'].notnull(), 'Status:'] =  [re.sub(r'</p>.*', "", x) for x in urban_turf.loc[urban_turf['Status:'].notnull(), 'Status:']]
urban_turf.loc[urban_turf['Address:'].notnull(), 'Status:'] =  [re.sub(r'</p>.*', "", x) for x in urban_turf.loc[urban_turf['Address:'].notnull(), 'Address:']]


'''

def get_lat_long(address):
    geolocator = Nominatim(user_agent="my_geocoder_app")
    time.sleep(1.1)
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return None
    
urban_turf['Latitude'] = ""
urban_turf['Longitude'] = ""

for i in range(0, len(urban_turf)):
    try:
        c = urban_turf['Address:'][i]
        d = re.sub("-[\d]*", "", c)
        a = get_lat_long(d)
        urban_turf['Latitude'][i] =  a[0]
        urban_turf['Longaitude'][i] =  a[1]

    except Exception:
        pass
urban_turf['No. of units:'] = pd.to_numeric(urban_turf['No. of units:'], errors='coerce')

'''
#urban_turf2 =urban_turf

C:\Users\annas\AppData\Local\Temp\ipykernel_13732\1471419196.py:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  labels = [item for item in page_body.findAll("p")]


'\n##########\nfor i in range(0, len(urban_turf.Address)):\n    if type(urban_turf["Pipeline Link"][i]) == str:\n    \n        page = requests.get(urban_turf["Pipeline Link"][i])\n        soup = BeautifulSoup(page.content, \'html.parser\')\n        page_body = soup.body\n        table = soup.find_all(\'div\', class_ = \'block\') \n        txt= str(table)\n        #every <span class> \n        #keep everything in between <span class="label">Project type:</span> and </p>\n\n        txt = txt.replace(\'\r\', \'\').replace(\'\n\', \'\')\n\n        #remove any instances where there are two spaces +\n        for k in range(0,100):\n            txt = txt.replace("  ", "")\n        #remove all new lines\n        txt = txt.split("<p>")  \n\n        txt = [x for x in txt if \'<span class="label">\' in x] #this is the problem\n        colnames = []\n        values = []\n\n        for j in range(0, len(txt)):\n            a = txt[j].split("</span>")\n\n            a[0] = re.sub(\'<span class="labe

In [576]:
#b = urban_turf['Website:'].notnull()
c = urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'][5]
#urban_turf.loc[c, 'Website:'] = [re.search(r'http.*\"\sstyle', x).group(0).replace('" style', "") for x in urban_turf.loc[c, 'Website:']]


for i in range(490, len(urban_turf)):
    if urban_turf['Website:'][i] == None:
        print(i)
    else:
        urban_turf['Website:'][i] = (re.search(r'[http]?[s]?www.*\"\sstyle', urban_turf['Website:'][i]).group(0).replace('" style', ""))

#urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'] = urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'].apply(website)

#urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'] = [re.search(r'http.*\"\sstyle', x).group(0).replace('" style', "") for x in urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:']]

490
491


AttributeError: 'NoneType' object has no attribute 'group'

In [588]:
#urban_turf['Website:'][i] = (re.search(r'[http]?[s]?www.*\"\sstyle', urban_turf['Website:'][i]).group(0).replace('" style', ""))
re.search(r'".*\"\sstyle', urban_turf['Website:'][495]).group(0).replace('" style', "").replace('"', "")
#urban_turf['Website:'][494]

'http://www.verdepointe.com/'

In [587]:
urban_turf['Website:'][495]

'<a href="http://www.verdepointe.com/" style="text-decoration: underline;" target="_blank" title="">www.verdepointe.com</a>\xa0\t\t\t\t\t\t\t\t</p>'

In [557]:
re.search(r'[https]?www.*\"\sstyle', a)

<re.Match object; span=(16, 47), match='www.forttottensquare.com" style'>

In [558]:
a

'<a href="http://www.forttottensquare.com" style="text-decoration: underline;" target="_blank" title="">www.forttottensquare.com</a>\xa0\t\t\t\t\t\t\t\t</p>'

In [ ]:
urban_turf['Website:'][485] = (re.search(r'www.*\"\sstyle', urban_turf['Website:'][485]).group(0).replace('" style', ""))


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
l = urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:']

In [445]:
re.search(r'http.*\"\sstyle', l).group(0).replace('" style', "")

'https://mcwb.com/properties/the-eave/'

In [266]:
def website(row):
    a = re.search(r'http.*\"\sstyle', row).group(0).replace('" style', "")
    return a

#b = urban_turf['Website:'].notnull()
#urban_turf.loc[b, 'Website:'] = urban_turf.apply(website, axis=1)


In [ ]:
c = re.search(r'http.*\"\sstyle', a).group(0).replace('" style', "")

KeyError: 'Website'

In [292]:
c = requests.get(urban_turf.iloc["Pipeline Link"][0])


TypeError: Cannot index by location index with a non-integer key

In [293]:
urban_turf.iloc["Pipeline Link"][0]

TypeError: Cannot index by location index with a non-integer key

In [297]:
urban_turf['Pipeline Link'][0]

'https://dc.urbanturf.com/pipeline/1308/The_Eave'

In [ ]:
c.gro

AttributeError: 'str' object has no attribute 'group'

In [174]:
urban_turf.loc[b, 'Architect:'] = [x.replace("\t", "").replace("</p>", "") for x in urban_turf.loc[b, 'Architect:']]

0      <a href="https://mcwb.com/properties/the-eave/...
1      <a href="https://mmgdevelopment.com/europa-818...
2      <a href="https://thecharlesmd.com/" style="tex...
3                                                   None
4                                                   None
                             ...                        
672    <a href="https://www.hendrixapts.com" style="t...
673    <a href="http://www.livewithlatitude.com" styl...
674    <a href="http://liveatthepearl.com" style="tex...
675    <a href="https://www.tenatclarendon.com" style...
676    <a href="http://www.esplanadenationalharbor.co...
Name: Website:, Length: 677, dtype: object

In [141]:
type(urban_turf['Architect:'][1])

str

In [106]:
#@urban_turf['Pipeline'] = urban_turf.apply(scrape_pipelinelink, axis=1)

a = scrape_pipelinelink(urban_turf.iloc[0])

for key, txt in a.items():
    if key in column_names:
        urban_turf[key][0] = a[key][0]

#for i in range(0, len(urban_turf)):
#    urban_turf['Pipeline'][i] = scrape(i)


C:\Users\annas\AppData\Local\Temp\ipykernel_13732\2662478429.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  urban_turf[key][0] = a[key][0]
C:\Users\annas\AppData\Local\Temp\ipykernel_13732\2662478429.py:7: FutureWarning: ChainedAssignme

In [107]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,The Eave,Adams Morgan,Condo,"Delivered, Selling",4 units,https://dc.urbanturf.com/pipeline/1308/The_Eave,None,4 </p>,Condos ...,4 </p>,...,None,"Starting from $399,900 ...","<a href=""https://mcwb.com/properties/the-eave/...",None,"November 4th, 2025 ...",2465 18th Street NW ...,"Delivered, Selling</p></div>, <div class=""bloc...",None,None,None
1,Europa,Capitol Hill,Rental,"Delivered, Leasing Up",49 units,https://dc.urbanturf.com/pipeline/733/Europa,None,49 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
2,The Charles,Bethesda,Rental,"Under Construction, Not Yet Selling",301 units,https://dc.urbanturf.com/pipeline/1119/The_Cha...,None,301 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1455-1457 Church Street NW,Logan Circle,Condo,"Under Construction, Not Yet Selling",65 units,https://dc.urbanturf.com/pipeline/1096/1455145...,None,65 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2001-2027 Martin Luther King Jr. Avenue,Anacostia,Townhomes & Condos,"Under Construction, Not Yet Selling",143 units,https://dc.urbanturf.com/pipeline/590/20012027...,None,143 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,45 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,https://dc.urbanturf.com/pipeline/320/Latitude...,None,265 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,507 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,https://dc.urbanturf.com/pipeline/368/Ten_at_C...,None,144 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None


In [94]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,The Eave,Adams Morgan,Condo,"Delivered, Selling",4 units,https://dc.urbanturf.com/pipeline/1308/The_Eave,None,4 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Europa,Capitol Hill,Rental,"Delivered, Leasing Up",49 units,https://dc.urbanturf.com/pipeline/733/Europa,None,49 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
2,The Charles,Bethesda,Rental,"Under Construction, Not Yet Selling",301 units,https://dc.urbanturf.com/pipeline/1119/The_Cha...,None,301 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1455-1457 Church Street NW,Logan Circle,Condo,"Under Construction, Not Yet Selling",65 units,https://dc.urbanturf.com/pipeline/1096/1455145...,None,65 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2001-2027 Martin Luther King Jr. Avenue,Anacostia,Townhomes & Condos,"Under Construction, Not Yet Selling",143 units,https://dc.urbanturf.com/pipeline/590/20012027...,None,143 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,45 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,https://dc.urbanturf.com/pipeline/320/Latitude...,None,265 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,507 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,https://dc.urbanturf.com/pipeline/368/Ten_at_C...,None,144 </p>,None,None,...,None,None,None,None,None,None,None,None,None,None


In [40]:
for i in range(0, len(urban_turf)):
    urban_turf['Project type:'][i] = scrape_pipelinelink(i)


C:\Users\annas\AppData\Local\Temp\ipykernel_13732\2544266255.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  urban_turf['Project type:'][i] = scrape_pipelinelink(i)


In [ ]:
urban_turf.apply(scrape, axis=1)

#df = df[df['col2'].eq('c').groupby(df['col1']).transform('sum').lt(2)]


#for i in range(0, len(urban_turf)):
#    urban_turf['Project type:'][i] = scrape_pipelinelink(i)



KeyError: "['The Eave', 'Adams Morgan', 'Condo', 'Delivered, Selling', '4  units', 'https://dc.urbanturf.com/pipeline/1308/The_Eave', None] not in index"

In [47]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,The Eave,Adams Morgan,Condo,"Delivered, Selling",4 units,https://dc.urbanturf.com/pipeline/1308/The_Eave,None,None,0,None,...,None,None,None,None,None,None,None,None,None,None
1,Europa,Capitol Hill,Rental,"Delivered, Leasing Up",49 units,https://dc.urbanturf.com/pipeline/733/Europa,None,None,1,None,...,None,None,None,None,None,None,None,None,None,None
2,The Charles,Bethesda,Rental,"Under Construction, Not Yet Selling",301 units,https://dc.urbanturf.com/pipeline/1119/The_Cha...,None,None,2,None,...,None,None,None,None,None,None,None,None,None,None
3,1455-1457 Church Street NW,Logan Circle,Condo,"Under Construction, Not Yet Selling",65 units,https://dc.urbanturf.com/pipeline/1096/1455145...,None,None,3,None,...,None,None,None,None,None,None,None,None,None,None
4,2001-2027 Martin Luther King Jr. Avenue,Anacostia,Townhomes & Condos,"Under Construction, Not Yet Selling",143 units,https://dc.urbanturf.com/pipeline/590/20012027...,None,None,4,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,None,672,None,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,https://dc.urbanturf.com/pipeline/320/Latitude...,None,None,673,None,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,None,674,None,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,https://dc.urbanturf.com/pipeline/368/Ten_at_C...,None,None,675,None,...,None,None,None,None,None,None,None,None,None,None


In [114]:
print(urban_turf['Pipeline Link'][544])

https://dc.urbanturf.com/pipeline/56/19Nineteen_Clarendon_Apartments


In [123]:
urban_turf = urban_turf.reset_index()

In [78]:
urban_turf['Project type:'][0] = scrape_pipelinelink(0)

In [111]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude,Project Type
0,The Eave,Adams Morgan,Condo,"Delivered, Selling",4 units,https://dc.urbanturf.com/pipeline/1308/The_Eave,None,None,4 </p>,4 </p>,...,"Starting from $399,900 ...","<a href=""https://mcwb.com/properties/the-eave/...",None,"November 4th, 2025 ...",2465 18th Street NW ...,"Delivered, Selling</p></div>, <div class=""bloc...",None,None,None,4 </p>
1,Europa,Capitol Hill,Rental,"Delivered, Leasing Up",49 units,https://dc.urbanturf.com/pipeline/733/Europa,None,None,49 </p>,49 </p>,...,None,"<a href=""https://mmgdevelopment.com/europa-818...",\t\t\t\t\tPGN Architects\t\t\t\t </p>,"November 3rd, 2025 ...",818 Potomac Avenue SE ...,"Delivered, Leasing Up</p>",2021 </p>...,None,None,4 </p>
2,The Charles,Bethesda,Rental,"Under Construction, Not Yet Selling",301 units,https://dc.urbanturf.com/pipeline/1119/The_Cha...,None,None,301 </p>,None,...,None,None,None,None,None,None,None,None,None,4 </p>
3,1455-1457 Church Street NW,Logan Circle,Condo,"Under Construction, Not Yet Selling",65 units,https://dc.urbanturf.com/pipeline/1096/1455145...,None,None,65 </p>,None,...,None,None,None,None,None,None,None,None,None,4 </p>
4,2001-2027 Martin Luther King Jr. Avenue,Anacostia,Townhomes & Condos,"Under Construction, Not Yet Selling",143 units,https://dc.urbanturf.com/pipeline/590/20012027...,None,None,143 </p>,None,...,None,None,None,None,None,None,None,None,None,4 </p>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4 </p>
674,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,https://dc.urbanturf.com/pipeline/320/Latitude...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4 </p>
675,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4 </p>
676,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,https://dc.urbanturf.com/pipeline/368/Ten_at_C...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4 </p>


In [71]:
a  =scrape_pipelinelink(0)
print(a)

                  4                </p>


In [68]:
urban_turf['Project Type'] = urban_turf.apply(lambda row: scrape_pipelinelink(row), axis=1)


KeyError: 'None of [Index([                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            \'The Eave\',\n                                                                                                                                                                                                                                                                                                                                                                                                                  

In [34]:
for key, txt in a.items():
        if key in column_names:
            urban_turf[key][0] = a[key][0]


In [58]:
urban_turf.apply(scrape_pipelinelink, axis=1)

KeyError: 'None of [Index([                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            \'The Eave\',\n                                                                                                                                                                                                                                                                                                                                                                                                                  

'                  4                </p>'

In [ ]:
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

#if there is a pipeline link

def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in list(len(txt))] #need to expand range?
    txt = [txt[i][1] for i in list(len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)

NameError: name 'urban_turf' is not defined

In [ ]:
def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in range(0, len(txt))] 
    txt = [txt[i][1] for i in range(0, len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

In [708]:
a[0]

'Project type:'

In [694]:
urban_turf['No. of units:'] = a[1]

In [29]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,The Eave,Adams Morgan,Condo,"Delivered, Selling",4 units,https://dc.urbanturf.com/pipeline/1308/The_Eave,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Europa,Capitol Hill,Rental,"Delivered, Leasing Up",49 units,https://dc.urbanturf.com/pipeline/733/Europa,None,None,Rental apart...,49 </p>,...,None,None,"<a href=""https://mmgdevelopment.com/europa-818...",\t\t\t\t\tPGN Architects\t\t\t\t </p>,"November 3rd, 2025 ...",818 Potomac Avenue SE ...,"Delivered, Leasing Up</p>",2021 </p>...,None,None
2,The Charles,Bethesda,Rental,"Under Construction, Not Yet Selling",301 units,https://dc.urbanturf.com/pipeline/1119/The_Cha...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1455-1457 Church Street NW,Logan Circle,Condo,"Under Construction, Not Yet Selling",65 units,https://dc.urbanturf.com/pipeline/1096/1455145...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2001-2027 Martin Luther King Jr. Avenue,Anacostia,Townhomes & Condos,"Under Construction, Not Yet Selling",143 units,https://dc.urbanturf.com/pipeline/590/20012027...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
674,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,https://dc.urbanturf.com/pipeline/320/Latitude...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
675,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
676,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,https://dc.urbanturf.com/pipeline/368/Ten_at_C...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


urban_turf

In [ ]:
def search_pipe_line(search, projecttype, status):
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + search + "&project_type=" + projecttype + "&status=" + status + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

In [ ]:
torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using value as keys
pattern = re.compile("|".join(torep.keys())) #joins

for i in range(0, len(list)):
    a = list[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #re.escape - remove extra values, #substitute the pattern in each value of the list (a)
    desc.append(text)